In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing necessary libraries


In [ ]:
import numpy as np
import pandas as pd
import tracemalloc
from copy import deepcopy
import time
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy.stats import mode
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support

## Algorithm Directory path

In [ ]:
# All the results are stored in this directory
directoryPath = '/content/drive/MyDrive/Colab Notebooks/Data Mining Assignments/Clustering/'

## K Means Clustering

In [ ]:
class KMeansClustering:

  def __init__(self, X, Y, K, dataType="numerical", attributeTypes = [], ranks = {}, numberOfIterations = 5, randomState = 24, symmetricity = {}):
    self.X = X
    self.Y = Y
    self.K = K
    self.clusterCenters = {}
    self.clusterLabels = None
    self.dataType = dataType
    self.attributeInfo = self.get_attribute_info(deepcopy(attributeTypes), ranks, symmetricity)
    self.randomState = randomState
    self.numberOfIterations = numberOfIterations
    self.withinClusterVariance = 0.0
    self.bCubedPrecision = 0.0
    self.bCubedRecall = 0.0
    self.silhouetteCoefficient = 0.0 
    
    if self.dataType == "mixed":
      assert len(attributeTypes) == self.X.shape[1], "If dataset type is mixed, then attributeType for each attribute needs to be passed."

  def get_withinClusterVariance(self):
    return self.withinClusterVariance

  def get_BCubedPrecision(self):
    return self.bCubedPrecision

  def get_BCubedRecall(self):
    return self.bCubedRecall

  def get_silhouetteCoefficient(self):
    return self.silhouetteCoefficient

  def train(self):
    minn = np.inf
    XNumerical, XNominal, XOrdinal = self.preprocess_X()
    for i in range(self.numberOfIterations):
      clusterCenters, withinClusterVariance, clusterLabels = self.trainOnce(XNumerical, XNominal, XOrdinal, i)
      # print("\nIteration : {} and withinClusterVariance : {}\n".format(i, withinClusterVariance))

      if withinClusterVariance < minn:
        minn = withinClusterVariance
        self.clusterCenters = clusterCenters
        self.clusterLabels = clusterLabels
        self.withinClusterVariance = minn

    self.silhouetteCoefficient = self.calculate_silhouette_coefficient()
    # print("\n withinClusterVariance : {}\n".format(self.withinClusterVariance))
    # print("\n silhouetteCoefficient : {}\n".format(self.silhouetteCoefficient))
    if len(self.Y):
      self.bCubedPrecision = self.calculate_bcubed_precision()
      self.bCubedRecall = self.calculate_bcubed_recall()
      # print("\n bCubedPrecision : {}\n".format(self.bCubedPrecision))
      # print("\n bCubedRecall : {}\n".format(self.bCubedRecall))

  def preprocess_X(self):
    XNumerical, XNominal, XOrdinal = np.array([]), np.array([]), np.array([])
    if self.dataType == 'numerical':
      XNumerical = self.X.astype(float)
      XNumerical = self.normalize_numerical_values(XNumerical)
      # print(XNumerical)
    
    elif self.dataType == 'nominal':
      XNominal = self.X
    
    elif self.dataType == 'ordinal':
      XOrdinal = self.X
      XOrdinal = self.format_ordinal_values(XOrdinal)
      # print(XOrdinal)

    elif self.dataType == 'mixed':
      XNumerical, XNominal, XOrdinal, _, _ = self.seperate_mixed_data()
      XNumerical = self.normalize_numerical_values(XNumerical) if len(XNumerical) else np.array([])
      XOrdinal = self.format_ordinal_values(XOrdinal) if len(XOrdinal) else np.array([])
    
    return XNumerical, XNominal, XOrdinal

  def trainOnce(self, XNumerical, XNominal, XOrdinal, iterationNum = 0, eps = 1e-6):
    withinClusterVariance = 0.0
    np.random.seed(self.randomState + iterationNum)
    clusterCentersIdx = np.random.choice(self.X.shape[0], self.K, replace=False)
    clusterCentersNumerical = XNumerical[clusterCentersIdx, :] if len(XNumerical) else np.array([])
    clusterCentersNominal = XNominal[clusterCentersIdx, :] if len(XNominal) else np.array([])
    clusterCentersOrdinal = XOrdinal[clusterCentersIdx, :] if len(XOrdinal) else np.array([])

    while True:
      ## eliminating square root from the euclidean distance calculation
      ## that means pairwise squared distances from cluster centers and the samples 
      pairwiseDistancesNumerical = 0 if len(XNumerical) == 0 else cdist(XNumerical, clusterCentersNumerical, metric='euclidean') ** 2
      pairwiseDistancesOrdinal = 0 if len(XOrdinal) == 0 else cdist(XOrdinal, clusterCentersOrdinal, metric='euclidean') ** 2  
      # hamming distance for nominal attributes
      pairwiseDistancesNominal = 0 if len(XNominal) == 0 else self.calculate_hamming_distance(XNominal, clusterCentersNominal) 

      pairwiseDistances = pairwiseDistancesNumerical + pairwiseDistancesNominal + pairwiseDistancesOrdinal
      # print("withinClusterVariance for iteration {} : {}".format(iterationNum, np.sum(np.min(pairwiseDistances, axis = 1))))
      clusterLabels = np.argmin(pairwiseDistances, axis = 1)

      anyCenterUpdated = False 

      for k in range(self.K):
        centerDifference = 0.0
        clusterObjectsIdx = (clusterLabels == k)
        newClusterCenterNumerical, newClusterCenterOrdinal, newClusterCenterNominal = np.array([]), np.array([]), np.array([])
        if len(XNumerical):
          singleClusterNumerical = XNumerical[clusterObjectsIdx, :]
          newClusterCenterNumerical = np.mean(singleClusterNumerical, axis = 0)
          centerDifference += np.linalg.norm(clusterCentersNumerical[k] - newClusterCenterNumerical)
        if len(XOrdinal):
          singleClusterOrdinal = XOrdinal[clusterObjectsIdx, :]
          newClusterCenterOrdinal = np.mean(singleClusterOrdinal, axis = 0)
          centerDifference += np.linalg.norm(clusterCentersOrdinal[k] - newClusterCenterOrdinal)
        if len(XNominal):
          singleClusterNominal = XNominal[clusterObjectsIdx, :]
          newClusterCenterNominal, _ = mode(singleClusterNominal, axis = 0)
          newClusterCenterNominal = np.squeeze(newClusterCenterNominal)
          centerDifference += (clusterCentersNominal[k] != newClusterCenterNominal).sum()

        # print(newClusterCenterNumerical)
        # print(newClusterCenterNominal)

        if centerDifference > eps:
          anyCenterUpdated = True
          if len(clusterCentersNumerical):
            clusterCentersNumerical[k] = newClusterCenterNumerical
          if len(clusterCentersOrdinal):
            clusterCentersOrdinal[k] = newClusterCenterOrdinal
          if len(clusterCentersNominal):
            clusterCentersNominal[k] = newClusterCenterNominal

      if not anyCenterUpdated:
        withinClusterVariance = np.sum(np.min(pairwiseDistances, axis = 1))
        break

    clusterCenters = {
        'numerical' : clusterCentersNumerical,
        'nominal' : clusterCentersNominal,
        'ordinal' : clusterCentersOrdinal
    }
    return clusterCenters, withinClusterVariance, clusterLabels

  def calculate_hamming_distance(self, X, Y):
    pairwiseHammingDistance = []
    for x in X:
      distance = (x != Y).astype(int).sum(axis = 1) 
      pairwiseHammingDistance.append(distance)

    return np.array(pairwiseHammingDistance)
      
  def normalize_numerical_values(self, XNumerical):
    mx, mn = np.array([]), np.array([])
    for attr in self.attributeInfo:
      if attr['type'] == 'numerical':
        mx = np.append(mx, attr['max'])
        mn = np.append(mn, attr['min'])

    # print(mn, mn)

    XNumerical = (XNumerical - mn) / (mx - mn)
    return XNumerical

  def format_ordinal_values(self, XOrdinal):
    ranks = []
    for attr in self.attributeInfo:
      if attr['type'] == 'ordinal':
        ranks.append(attr['rank'])

    for j in range(XOrdinal.shape[1]):
      rank = ranks[j]
      if len(rank):
        for i in range(XOrdinal.shape[0]):
          XOrdinal[i, j] = rank[XOrdinal[i, j]]

    XOrdinal = XOrdinal.astype(float)

    # scaling each attribute in the range [0, 1]
    for j in range(XOrdinal.shape[1]):
      XOrdinal[:, j] = (XOrdinal[:, j] - 1)/(np.max(XOrdinal[:, j]) - 1)
    
    return XOrdinal

  def get_attribute_info(self, attributeTypes, ranks, symmetricity):
    attributeInfo = []
    
    for i in range(self.X.shape[1]):
      column = self.X[:, i]
      distinctValues = np.unique(column)
      if len(attributeTypes) <= i:
        attributeTypes.append(self.dataType)

      attributeInfo.append({
          'idx' : i,
          'type' : attributeTypes[i],
          'distinctValues' : distinctValues if attributeTypes[i] != 'numerical' else None,
          'max' : np.amax(column.astype(float)) if attributeTypes[i] == 'numerical' else None,
          'min' : np.amin(column.astype(float)) if attributeTypes[i] == 'numerical' else None,
          'rank' : ranks[i] if i in ranks else {},
          'symmetricity' : symmetricity[i] if i in symmetricity else None
      })

    return np.array(attributeInfo)

  def seperate_mixed_data(self):
    attributeTypes = np.array([])
    symmetricIdxs, asymmetricIdxs = np.array([]), np.array([])
    for attr in self.attributeInfo:
      attributeTypes = np.append(attributeTypes, attr['type'])
      if attr['type'] == 'binary' and attr['symmetricity'] == 'symmetric':
        symmetricIdxs = np.append(symmetricIdxs, attr['idx'])
      elif attr['type'] == 'binary' and attr['symmetricity'] == 'asymmetric':
        asymmetricIdxs = np.append(asymmetricIdxs, attr['idx'])
    
    XNumerical = self.X[:, attributeTypes == 'numerical']
    XNominal = self.X[:, attributeTypes == 'nominal']
    XOrdinal = self.X[:, attributeTypes == 'ordinal']
    XBinarySymmetric = self.X[:, symmetricIdxs] if len(symmetricIdxs) else np.array([])
    XBinaryAsymmetric = self.X[:, asymmetricIdxs] if len(asymmetricIdxs) else np.array([])

    XNumerical = XNumerical.astype(float)
    # XBinarySymmetric = XBinarySymmetric.astype(int)
    # XBinaryAsymmetric = XBinaryAsymmetric.astype(int)

    return XNumerical, XNominal, XOrdinal, XBinarySymmetric, XBinaryAsymmetric

  def calculate_pairwise_distances(self, XNumerical, XOrdinal, XNominal, ObjectsIdx1, objectsIdx2):
    ## eliminating square root from the euclidean distance calculation
    ## that means pairwise squared distances from cluster centers and the samples 
    pairwiseDistancesNumerical = 0 if len(XNumerical) == 0 else cdist(XNumerical[ObjectsIdx1, :], XNumerical[objectsIdx2, :], metric='euclidean') ** 2
    pairwiseDistancesOrdinal = 0 if len(XOrdinal) == 0 else cdist(XOrdinal[ObjectsIdx1, :], XOrdinal[objectsIdx2, :], metric='euclidean') ** 2  
    # hamming distance for nominal attributes
    pairwiseDistancesNominal = 0 if len(XNominal) == 0 else self.calculate_hamming_distance(XNominal[ObjectsIdx1, :], XNominal[objectsIdx2, :]) 

    pairwiseDistances = pairwiseDistancesNumerical + pairwiseDistancesNominal + pairwiseDistancesOrdinal
    return np.sqrt(pairwiseDistances)

  def calculate_silhouette_coefficient(self):
    s = np.array([])
    XNumerical, XNominal, XOrdinal = self.preprocess_X()
    for i in range(self.X.shape[0]):
      minn = np.inf
      for k in range(self.K):
        clusterObjectsIdx = (self.clusterLabels == k)
        pairwiseDistances = self.calculate_pairwise_distances( XNumerical, XOrdinal, XNominal, np.array([i]), clusterObjectsIdx)
        if self.clusterLabels[i] == k:
          avgDistance = 0.0 if clusterObjectsIdx.sum() == 1 else pairwiseDistances.sum() / (clusterObjectsIdx.sum()-1)
          a = avgDistance
        else:
          avgDistance = pairwiseDistances.sum() / clusterObjectsIdx.sum()
          minn = min(minn, avgDistance)
      b = minn
      s = np.append(s, (b-a) / max(a,b) )

    return np.mean(s)  

  def calculate_bcubed_precision(self):
    bCubedPrecision = 0.0
    for i in range(self.X.shape[0]):
      clusterObjectsIdx = (self.clusterLabels == self.clusterLabels[i])
      correctNess = (self.Y[clusterObjectsIdx] == self.Y[i]).sum()
      avgCorrectNess = 0 if clusterObjectsIdx.sum() == 1 else (correctNess-1)/ (clusterObjectsIdx.sum()-1)
      bCubedPrecision += avgCorrectNess

    return bCubedPrecision/ self.X.shape[0]

  def calculate_bcubed_recall(self):
    bCubedRecall = 0.0
    for i in range(self.X.shape[0]):
      sameClassObjectsIdx = (self.Y == self.Y[i])
      correctNess = (self.clusterLabels[sameClassObjectsIdx] == self.clusterLabels[i]).sum()
      avgCorrectNess = 0 if sameClassObjectsIdx.sum() == 1 else (correctNess-1)/ (sameClassObjectsIdx.sum()-1)
      bCubedRecall += avgCorrectNess
      
    return bCubedRecall/ self.X.shape[0]

  def predict_one(self, x):
    return

  def predict(self, XTest):
    return

In [ ]:
def plot_cluster(X, K, clusterCenters, clusterLabels):
    color = ["red","green", "blue", "yellow", "black"]
    for k in range(K):
        plt.scatter(X[clusterLabels == k, 0], X[clusterLabels == k, 1], color=color[k])
    
    plt.scatter(clusterCenters[:, 0] , clusterCenters[:, 1], color="black")
    plt.show()

##Running on Sample datasets


In [ ]:
# dName = 'sampleDataset-1.csv'
# filePath = '{}Datasets/{}'.format(directoryPath, dName)
# df = pd.read_csv(filePath, sep=",", header=None)

# X = df.to_numpy()
# Y = np.array([])
# print('instances = {}, features= {} '.format(X.shape[0], X.shape[1]))

# kMeansClustering = KMeansClustering(X, Y, K = 4, dataType="numerical")
# kMeansClustering.train()
# plot_cluster(kMeansClustering.normalize_numerical_values(X), 4, kMeansClustering.clusterCenters["numerical"], kMeansClustering.clusterLabels)

In [ ]:
# dName = 'sampleDataset-2.csv'
# filePath = '{}Datasets/{}'.format(directoryPath, dName)
# df = pd.read_csv(filePath, sep=",", header=None)

# X = df.to_numpy()
# Y = np.array([])
# print('instances = {}, features= {} '.format(X.shape[0], X.shape[1]))

# kMeansClustering = KMeansClustering(X, Y, K = 4, dataType="numerical")
# kMeansClustering.train()

In [ ]:
# from sklearn.datasets import load_iris
# X, Y = load_iris(return_X_y= True)
# Y = np.squeeze(Y)
# # print(Y)
# # print('instances = {}, features= {} '.format(X.shape[0], X.shape[1]))

# kMeansClustering = KMeansClustering(X, Y, K = 3, dataType="numerical")
# kMeansClustering.train()
# # print(kMeansClustering.withinClusterVariance)
# # print(kMeansClustering.clusterLabels)